In [1]:
import gensim
import spacy
from gensim import corpora
import nltk
from nltk.corpus import stopwords
import pandas as pd
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import string

In [2]:
pre_dataset_train_cnn= pd.read_csv("dataset_train_CNN")
pre_dataset_train_cnn.shape


(28712, 3)

In [3]:
reduced_data = pre_dataset_train_cnn.sample(frac=0.5, random_state=42)

In [4]:
reduced_data.shape

(14356, 3)

In [10]:
import numpy as np
from gensim import corpora, models
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(reduced_data.iloc[0]['article'])

# Tokenize sentences 
tokenized_sentences = [sentence.lower().split() for sentence in sentences]

# Create dictionary and corpus for LDA
dictionary = corpora.Dictionary(tokenized_sentences)
corpus = [dictionary.doc2bow(tokens) for tokens in tokenized_sentences]

# Build LDA model
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

# Function to get dominant topic of a sentence
def get_dominant_topic(sentence):
    bow = dictionary.doc2bow(sentence.lower().split())
    topic_distribution = lda_model.get_document_topics(bow)
    dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]
    return dominant_topic

# Target dominant topic (example)
target_topic = 3

# Function to filter sentences based on topic similarity
def filter_sentences(sentences, target_topic):
    filtered_sentences = []
    for sentence in sentences:
        dominant_topic = get_dominant_topic(sentence)
        if dominant_topic == target_topic:
            filtered_sentences.append(sentence)
    return filtered_sentences

# Filter sentences based on the target dominant topic
filtered_sentences = filter_sentences(sentences, target_topic)

# Print the filtered sentences
print("Sentences related to topic", target_topic)
for sentence in filtered_sentences:
    print("-", sentence)


Sentences related to topic 3
- Few options: Sundby (taking part in an 800-mile .
- 'I am a determined fighter and extremely .
- affordable, lifesaving medical insurance policy has been canceled .
- Think Progress argued that United selfishly 'packed .
- USA Today oped editor David Mastio took issue with that assessment.
- president's promise, "You can keep your health plan"?
- that "You can keep your doctor"?'
- He called Pfeiffer a 'fast-talking flack' and Think Progress – the house blog of a group run by former Clinton White House chief of staff John Podesta – 'a leftist propaganda outfit.'


In [11]:
print(len(sentences), len(filtered_sentences))

61 8


In [12]:
topic_names = {}

for i, topic in lda_model.show_topics(formatted=False):
    top_words = [word for word, _ in topic]
    topic_name = " ".join(top_words)
    topic_names[f"Topic {i+1}"] = topic_name


pprint(topic_names)

{'Topic 1': 'the her and a that for as plan she of',
 'Topic 2': 'the that her to . is cancer a in and',
 'Topic 3': 'to a the . with and insurance of it an',
 'Topic 4': '. that a of can "you your keep think progress',
 'Topic 5': 'the to and a of insurance . have is my'}


In [13]:
def preprocess(text):
    tokens = nltk.word_tokenize(text.lower())

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]

    return filtered_tokens

In [14]:
preprocessed_topics = [preprocess(doc) for doc in topic_names.values()]
preprocessed_topics

[['plan'],
 ['cancer'],
 ['insurance'],
 ['keep', 'think', 'progress'],
 ['insurance']]